In [31]:
#imports
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import healpy as hp
import scipy.stats as st

In [32]:
def cal_power_spectrum_mc(lmax,D,theta):
    #power law, exclude the monopole later on, mc==model clean
    l = np.arange(lmax + 1)
    # calculate model Als according to a power law
    return D * (l ** theta)   
    

def getlm(lmax):
    halflen = hp.Alm.getsize(lmax)
    l1, m1 = hp.Alm.getlm(lmax, np.arange(halflen))
    l_neg = l1[m1 > 0][::-1]
    m_neg = -m1[m1 > 0][::-1]
    l_full = np.concatenate((l_neg, l1))
    m_full = np.concatenate((m_neg, m1))
    return l_full, m_full
        
    
def getlmidx(lmax,ll):
    #input: lmax, the l that we want to find all (l,m) index
    mcount = lmax * (lmax + 1) // 2
    lmidx = hp.Alm.getidx(lmax, ll, np.arange(ll + 1))
    flipped_idx = np.flip(mcount - (lmidx[1:] - lmax))
    truelmidx = np.concatenate((flipped_idx, lmidx + mcount)).astype(int)
    return truelmidx

#both definitions above from Kate

def getlmidx_v2(lmax,ll, mm): 
    #input: lmax, the l and m that we want to find single (l,m) index
    l,m = getlm(lmax)
    for i in range((lmax+1)**2):
        if ll==l[i] and mm==m[i]:\
            return i

In [33]:
def draw_alms_blms_v4(Al, Bl, Cl, lmax):
    l, m = hp.Alm.getlm(lmax)
    size = (lmax + 1) ** 2
    
    # Prepare output arrays\n",
    alms = np.zeros(size, dtype=complex)
    blms = np.zeros(size, dtype=complex)
    
    # Identify indices for real and imaginary components
    real_mask = l > 0
    imag_mask = m > 0
    
    lr, mr = l[real_mask], m[real_mask]
    li, mi = l[imag_mask], m[imag_mask]
   
    # Precompute covariance matrices\n",
    def compute_cov(A, B, C, half=False):
        factor = 0.5 if half else 1.0
        return [[factor * A, factor * C], [factor * C, factor * B]]
    
    # Generate real components
    for i, (ll, mm) in enumerate(zip(lr, mr)):
        half = mm != 0
        cov_r = compute_cov(Al[ll], Bl[ll], Cl[ll], half)
        ar, br = np.random.multivariate_normal([0, 0], cov_r)
        ai = bi = 0
    
        # Handle imaginary components if applicable
        if mm > 0:
            cov_i = compute_cov(Al[ll], Bl[ll], Cl[ll], True)
            ai, bi = np.random.multivariate_normal([0, 0], cov_i)
   
            # Assign values for +m
            idx = getlmidx_v2(lmax, ll, mm)
            alms[idx] = complex(ar, ai)
            blms[idx] = complex(br, bi)
  
        # Assign conjugate values for -m
        if mm != 0:
            idx_neg = getlmidx_v2(lmax, ll, -mm)
            sign = (-1) ** mm
            alms[idx_neg] = np.conj(alms[idx]) * sign
            blms[idx_neg] = np.conj(blms[idx]) * sign
  
    return alms, blms

In [34]:
def C2Rcov(covmat, lmax):
    #input: complex covariance matrix
    size = len(covmat)
    ll, mm = getlm(lmax)
    
    Rcovmat = np.zeros((2 * size, 2 * size), dtype=complex)

    # Precompute indices for efficiency
    iip = np.where(mm == 0, np.arange(size), size - 1 - np.arange(size))
    jjp = np.where(mm == 0, np.arange(size), size - 1 - np.arange(size))
    
    # Precompute (-1)^|m| values
    neg_q = (-1) ** np.abs(mm)
    neg_m = (-1) ** np.abs(mm)
  
    for ii in range(size):
        for jj in range(size):
            q, m = mm[ii], mm[jj]
            iip_idx, jjp_idx = iip[ii], jjp[jj]
   
            term1 = covmat[ii, jj]
            term2 = neg_q[ii] * covmat[iip_idx, jj]
            term3 = neg_m[jj] * covmat[ii, jjp_idx]
            term4 = neg_q[ii] * neg_m[jj] * covmat[iip_idx, jjp_idx]
  
            Rcovmat[ii, jj] = 0.25 * (term1 + term2 + term3 + term4)   # R_pq R_lm
            Rcovmat[ii + size, jj + size] = 0.25 * (term1 - term2 - term3 + term4)  # I_pq I_lm
            Rcovmat[ii, jj + size] = -0.25j * (term1 + term2 - term3 - term4)  # R_pq I_lm,
            Rcovmat[ii + size, jj] = 0.25j * (term1 - term2 + term3 - term4)   # I_pq R_lm
 
        return Rcovmat


def covcrosscl(blm, gwcov, lmax):
    #input: blm's, GW alm cov mat, output: Cl covmat\n",
    Rgwcov = C2Rcov(gwcov, lmax)  #take the NxN complex covariance matrix and create a 2Nx2N Real covariance matrix
    size = len(gwcov)
    covcl = np.zeros((lmax + 1, lmax + 1))
    
    # Precompute indices and scaled blm values
    precomputed = {}
    for l in range(lmax + 1):
        lmidx = getlmidx(lmax, l)
        factor = 1.0 / (2 * l + 1)
        A = np.real(blm[lmidx]) * factor
        B = np.imag(blm[lmidx]) * factor
        idx = np.concatenate((lmidx, lmidx + size))
        vec = np.concatenate((A, B))
        precomputed[l] = (idx, vec)
    
    # Compute the covariance
    for l1 in range(lmax + 1):
        idx1, vec1 = precomputed[l1]
        for l2 in range(lmax + 1):
            idx2, vec2 = precomputed[l2]
            SS = Rgwcov[np.ix_(idx1, idx2)]
            covcl[l1, l2] = np.dot(vec1, np.dot(SS, vec2))
   
    return covcl

In [35]:
def constructCrossPower(alms, blms, lmax):
    crossPower = np.zeros(lmax + 1)
    factors = 1 / (2 * np.arange(lmax + 1) + 1)  
    # Precompute normalization factors
    for ll in range(lmax + 1):
        idx = getlmidx(lmax, ll)
        crossPower[ll] = factors[ll] * np.real(np.vdot(alms[idx], blms[idx]))
    
    return crossPower

def getDirtyModelCrossPower(amplitude, exponent, gw_powerModelClean, em_powerModelClean, fisher, lmax):
    # Generate the clean cross power spectrum
    crossPowerClean = cal_power_spectrum_mc(lmax, amplitude, exponent)
   
    # Generate clean alms and blms\n",
    alms_clean, blms_clean = draw_alms_blms_v4(gw_powerModelClean, em_powerModelClean, crossPowerClean, lmax)
   
    # Apply the Fisher matrix to get dirty alms and blms
    alms_dirty = fisher @ alms_clean
    blms_dirty = fisher @ blms_clean
    
    # Construct the dirty cross power spectrum
    crossPowerDirty = constructCrossPower(alms_dirty, blms_dirty, lmax)

    return crossPowerDirty, blms_dirty

def getDirtyInjectedCrossPower(gw_powerModelClean, em_powerModelClean, crossPowerSignal, fisher, gw_noise, lmax):
    alms_modelClean, blms_modelClean = draw_alms_blms_v4(gw_powerModelClean, em_powerModelClean, crossPowerSignal, lmax)
    alms_modelDirty, blms_modelDirty = np.matmul(fisher,alms_modelClean), np.matmul(fisher,blms_modelClean)
    alms_injectedDirty = alms_modelDirty + gw_noise
    crossPowerInjectedDirty = constructCrossPower(alms_injectedDirty, blms_modelDirty, lmax)
    return crossPowerInjectedDirty



def getNoiseFromCovmat(covmat, lmax): #specifically for spherical harmonic coefficients
    real_covmat = C2Rcov(covmat, lmax)
    means = np.zeros(2*((lmax+1)**2))
    rv = np.random.multivariate_normal(mean=means, cov=real_covmat)
    
    real_alms = rv[:(lmax+1)**2]
    imaginary_alms = rv[(lmax+1)**2:]
    
    noise_alms = np.zeros((lmax+1)**2, dtype=complex)
    for i in range((lmax+1)**2):
        noise_alms[i] = complex(real_alms[i], imaginary_alms[i])
    
    return noise_alms


def getDirtyModelCrossPowerMeanCovmatTable(amplitude_range, step_size, exponent, gw_powerModelClean, em_powerModelClean, fisher, lmax, nsim):
    # Generate amplitude list using numpy's efficient arange
    amplitude_list = np.arange(amplitude_range[0], amplitude_range[1] + step_size, step_size)

    # Pre-allocate arrays
    crossPowerModelDirty_all = np.zeros((len(amplitude_list), nsim, lmax + 1))

    # Vectorized computation for simulations
    for i, amplitude in enumerate(amplitude_list):
        for n in range(nsim):
            crossPowerModelDirty, _ = getDirtyModelCrossPower(amplitude, exponent, gw_powerModelClean, em_powerModelClean, fisher, lmax)
            crossPowerModelDirty_all[i, n] = crossPowerModelDirty

    # Efficient aggregation using numpy functions
    crossPowerModelDirty_means = np.mean(crossPowerModelDirty_all, axis=1)
    crossPowerModelDirty_std = np.std(crossPowerModelDirty_all, axis=1)
    crossPowerModelDirty_sigma = np.array([np.cov(crossPowerModelDirty_all[i], rowvar=False) for i in range(len(amplitude_list))])

    return crossPowerModelDirty_means, crossPowerModelDirty_std, crossPowerModelDirty_sigma


In [36]:
########## Set parameters ######################
l_max = 6 #max angular resolution
nsim = 1000 #number of times we draw the alms/blms from the model
exp = 1 # exponent in model 

#calculate model clean gw power spectrum
gw_amp = 5e-97
Al_mc = cal_power_spectrum_mc(l_max,gw_amp,exp)

#calculate model clean em power spectrum
em_amp = 8e-101
Bl_mc = cal_power_spectrum_mc(l_max,em_amp,exp)
  
#calculate model clean gwem power spectrum amp limits
gwem_amp_max = np.sqrt(gw_amp*em_amp) # goes from inverse correlation (rho=-1) to correlation (rho=1)
amp_range = [-gwem_amp_max, gwem_amp_max]
    
num_steps = 202
step_size = (amp_range[1]-amp_range[0])/num_steps
    
############## Draw Models into Dirty Space ###########
  
#import Fisher matrix
Fisher = np.load('data/gw_fisher_lmax6_allfreq_HL.npy')
  
#draw model into dirty space once
Cl_md,_ = getDirtyModelCrossPower(gwem_amp_max, exp, Al_mc, Bl_mc, Fisher, l_max)

#get table of models in the dirty space at each amplitude interested
Cl_md_means, Cl_md_stds, Cl_md_sigmas = getDirtyModelCrossPowerMeanCovmatTable(amp_range, step_size, exp, Al_mc, Bl_mc, Fisher, l_max, nsim)
  
np.save("output/ClMeanDraw.npy", Cl_md_means)
np.save("output/ClStdDraw.npy", Cl_md_stds)
np.save("output/ClSigmaDraw.npy", Cl_md_sigmas)

########### Inject Signal into Dirty Space ##########
#calculate model clean cross-power signal
gwem_signal = 3e-99
Cl_signal = cal_power_spectrum_mc(l_max,gwem_signal,exp)

#calculate gw noise to add to alms (in dirty space)
gw_noise = getNoiseFromCovmat(Fisher, l_max)

#draw signal into dirty space + add O3 noise\n",
Cl_injd = getDirtyInjectedCrossPower(Al_mc, Bl_mc, Cl_signal, Fisher, gw_noise, l_max)
############ Parameter Estimation ####################

Cl_means = np.load("output/ClMeanDraw.npy")
Cl_stds = np.load("output/ClStdDraw.npy")
Cl_sigmas = np.load("output/ClSigmaDraw.npy")

/var/folders/63/39kkwt414rv8gx91d72hdt240000gn/T/ipykernel_96872/3910043584.py:39: ComplexWarning: Casting complex values to real discards the imaginary part
  rv = np.random.multivariate_normal(mean=means, cov=real_covmat)
/var/folders/63/39kkwt414rv8gx91d72hdt240000gn/T/ipykernel_96872/3910043584.py:39: RuntimeWarning: covariance is not symmetric positive-semidefinite.
  rv = np.random.multivariate_normal(mean=means, cov=real_covmat)
